## Set Params

In [1]:
img_path = r'C:/Users/Path_to_image_for_inference.tif'

out_path = r'C:/Users/Path_to_folder_for_output/' #inference output is full FSD map
site = "newV1_2019ort_" #give descriptive name for output

mod = "v1" #enter "v1" for original model, or "v2" for updated model with padding
path_to_weights = r'C:/Users/Path_to_model_weights/'

rgba_out = r'C:/Users/Path_to_folder_for_output_tiles/' #output are stacked image tiles RGB+heatmap
#rgba tiles are saved to folders labeled with predicted tree count

NoData = -9999 #enter NoData value of the incoming img
tile_size = 128 #enter desired tile size between 64 - 128, default 128
batch_size = 100 #enter batch size, max 250, default 100

minval = 0 #enter min val for normalization, default 0
maxval = 255 #enter max val for normalization, default 255

generate_heatmap = True #enter 'False' or 'True'
heatmap_algo = "cgc" #enter 'gc' for Grad-CAM, 'cgc' for custom Grad-CAM, or 'gcpp' for Grad-CAM++
target_layer1 = "batch_normalization_23" #conv layer you want to view activations for
target_layer2 = "batch_normalization_13" #conv layer you want to view activations for
           
normalize_heatmap = "local" #enter 'none', 'local', or 'global'
generate_rgba = False #enter True to output RGB+heatmap tiles, default False

from ipynb.fs.full.TreeCRowNNv2_Functions_2  import *


# Set Up

In [2]:
import os, sys, PIL,time,ipynb.fs, cv2
import numpy as np
import tifffile as tiff
from PIL import Image
Image.MAX_IMAGE_PIXELS = 5000000000

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow,figure
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from keras import backend as K

print(tf.__version__)
print(f"Tensorflow ver. {tf.__version__}")
physical_device = tf.config.experimental.list_physical_devices('CPU')
print(f'Device found : {physical_device}')
K.clear_session()

from ipynb.fs.full.TreeCRowNNv2_Functions_2 import *

2.8.0
Tensorflow ver. 2.8.0
Device found : [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Import Data

In [11]:
padding = int((128-tile_size)/2)
print(f"padding : {padding}")
k1=3
k2=7
k3=15

padding : 0


In [12]:
image = tiff.imread(img_path)
image[image==NoData] = 0 #np.nan #change this to nan if desired
img = image[:,:,:3]
print(img.min(),img.max())

for i in range(0,2):
    crop_image(img[:,:,i],tile_size)

print(img.min(),img.max(),img.shape)

0 255
47 59951 39 36391
47 59951 39 36391
0 255 (36430, 59998, 3)


In [ ]:
plt.imshow(img/255) #to avoid generating error, only view if the image is small

In [13]:
y,x,chan = img.shape
big_x = round(int(x/tile_size),0)
cropx = (big_x*tile_size)
big_y = round(int(y/tile_size),0)
cropy = (big_y*tile_size)

In [14]:
blank_img = np.zeros([big_y,big_x],dtype=int)
blank_img2 = np.zeros([y,x],dtype=np.float32)
print(blank_img.shape,blank_img2.shape)

(284, 468) (36430, 59998)


## Load Model and Weights

In [16]:
NN_name='TreeCRowNN'
#architecture of model v1 and v2 are the same, only weights are different
def tc_model(lr,spe,u,u2,u3,k1,k2,k3):
    import time
    import tensorflow as tf
    from tensorflow import keras
    from keras import backend as K
    from keras import models,layers
    from tensorflow.keras.layers import Input, Conv2D,Conv1D, UpSampling2D,GlobalMaxPool2D,GlobalAveragePooling2D, concatenate,Dense, Flatten, Dropout,BatchNormalization, MaxPooling2D
    from tensorflow.keras.models import Model, Sequential, load_model
    from tensorflow.keras.optimizers import Adam
    
    in1 = Input(shape=(128,128,3))
    
    conv1 = Conv2D(u,(k1,k1),activation='relu', padding='same')(in1)
    #conv1.trainable = False
    BN1 = BatchNormalization()(conv1)
    #BN1.trainable = False
    conv2 = Conv2D(u,(k1,k1),activation='relu', padding='same')(BN1)
    #conv2.trainable = False
    BN2 = BatchNormalization()(conv2)
    #BN2.trainable = False
 
    small1 = Conv2D(u,(k1,k1), activation='relu', padding='same')(BN2)
    #small1.trainable = False
    BN3 = BatchNormalization()(small1)
    #BN3.trainable = False
    small2 = Conv2D(u,(k1,k1), activation='relu', padding='same')(BN3)
    #small2.trainable = False
    BN4 = BatchNormalization()(small2)
    #BN4.trainable = False
    small3 = MaxPooling2D((2,2))(BN4)
    #small3.trainable = False
    small4 = Dropout(0.2)(small3)
    
    small5 = Conv2D(u,(k1,k1), activation='relu',padding='same')(small4)
    #small5.trainable = False
    BN5 = BatchNormalization()(small5)
    #BN5.trainable = False
    small6 = Conv2D(u,(k1,k1), activation='relu',padding='same')(BN5)
    #small6.trainable = False
    BN6 = BatchNormalization()(small6)
    #BN6.trainable = False
    small7 = Dropout(0.2)(BN6)
    
    small8 = Conv2D(u,(k1,k1), activation='relu',padding='same')(small7)
    #small8.trainable = False
    BN7 = BatchNormalization()(small8)
    #BN7.trainable = False
    small9 = Conv2D(u,(k1,k1), activation='relu',padding='same')(BN7)
    #small9.trainable = False
    BN8 = BatchNormalization()(small9)
    #BN8.trainable = False
    small10 = MaxPooling2D((2,2))(BN8)
    #small10.trainable = False
    small11 = Dropout(0.2)(small10)

    med1 = Conv2D(u,(k2,k2), activation='relu', padding='same')(BN2)
    #med1.trainable = False
    BN9 = BatchNormalization()(med1)
    #BN9.trainable = False
    med2 = Conv2D(u,(k2,k2), activation='relu', padding='same')(BN9)
    #med2.trainable = False
    BN10 = BatchNormalization()(med2)
    #BN10.trainable = False
    med3 = MaxPooling2D((2,2))(BN10)
    #med3.trainable = False
    med4 = Dropout(0.2)(med3)
    
    med5 = Conv2D(u,(k2,k2), activation='relu',padding='same')(med4)
    #med5.trainable = False
    BN11 = BatchNormalization()(med5)
    #BN11.trainable = False
    med6 = Conv2D(u,(k2,k2), activation='relu',padding='same')(BN11)
    #med6.trainable = False
    BN12 = BatchNormalization()(med6)
    #BN12.trainable = False
    med7 = Dropout(0.2)(BN12)
    
    med8 = Conv2D(u,(k2,k2), activation='relu',padding='same')(med7)
    #med8.trainable = False
    BN13 = BatchNormalization()(med8)
    #BN13.trainable = False
    med9 = Conv2D(u,(k2,k2), activation='relu',padding='same')(BN13)
    #med9.trainable = False
    BN14 = BatchNormalization()(med9)
    #BN14.trainable = False
    med10 = MaxPooling2D((2,2))(BN14)
    #med10.trainable = False
    med11 = Dropout(0.2)(med10)

    big1 = Conv2D(u,(k3,k3), activation='relu', padding='same')(BN2)
    #big1.trainable = False
    BN15 = BatchNormalization()(big1)
    #BN15.trainable = False
    big2 = Conv2D(u,(k3,k3), activation='relu', padding='same')(BN15)
    #big2.trainable = False
    BN16 = BatchNormalization()(big2)
    #BN16.trainable = False
    big3 = MaxPooling2D((2,2))(BN16)
    #big3.trainable = False
    big4 = Dropout(0.2)(big3)
    
    big5 = Conv2D(u,(k3,k3), activation='relu',padding='same')(big4)
    #big5.trainable = False
    BN17 = BatchNormalization()(big5)
    #BN17.trainable = False
    big6 = Conv2D(u,(k3,k3), activation='relu',padding='same')(BN17)
    #big6.trainable = False
    BN18 = BatchNormalization()(big6)
    #BN18.trainable = False
    big7 = Dropout(0.2)(BN18)
    
    big8 = Conv2D(u,(k3,k3), activation='relu',padding='same')(big7)
    #big8.trainable = False
    BN19 = BatchNormalization()(big8)
    #BN19.trainable = False
    big9 = Conv2D(u,(k3,k3), activation='relu',padding='same')(BN19)
    #big9.trainable = False
    BN20 = BatchNormalization()(big9)
    #BN20.trainable = False
    big10 = MaxPooling2D((2,2))(BN20)
    #big10.trainable = False
    big11 = Dropout(0.2)(big10)

    concat1 = tf.keras.layers.Concatenate()([small11,med11,big11])

    FC1 = Conv2D(u2,(1,1), activation='relu',padding='valid')(concat1)
    BN21 = BatchNormalization()(FC1)
    FC2 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN21)
    BN22 = BatchNormalization()(FC2)
    FC3 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN22)
    BN23 = BatchNormalization()(FC3)
    FC4 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN23)
    BN24 = BatchNormalization()(FC4)

    
    flat = Flatten()(BN24)

    dense1 = keras.layers.Dense(u3, activation='relu')(flat)
    BN25 = BatchNormalization()(dense1)
    dense2 = keras.layers.Dense(u3, activation='relu')(BN25)
    BN26 = BatchNormalization()(dense2)
    dense3 = keras.layers.Dense(u3, activation='relu')(BN26)
    BN27 = BatchNormalization()(dense3)
    dense4 = keras.layers.Dense(u3, activation='relu')(BN27)
    BN28 = BatchNormalization()(dense4)
    dense5 = keras.layers.Dense(u3, activation='relu')(BN28)
    BN29 = BatchNormalization()(dense5)
    out1 = keras.layers.Dense(1, activation='linear')(BN29)

    model = Model(inputs=[in1], outputs=[out1])

    model.compile(loss="MeanAbsoluteError", 
              optimizer =keras.optimizers.Adam(learning_rate=lr),
              metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError()])

    return model


In [17]:
#load best model and evaluate
if mod = "v1":
    lr,spe,u,u2,u3 = 0.001,160,16,16,16
    weights = os.path.join(path_to_weights+"Weights_v1.h5")
if mod = "v2":
    lr,spe,u,u2,u3 = 0.001,160,16,12,12
    weights = os.path.join(path_to_weights+"Weights_v2.h5")

model=tc_model(lr,spe,u,u2,u3,k1,k2,k3)
model.load_weights(weights)

## Perform Batch Inference

In [ ]:
#tree_counts2,T2 = create_image2(img,blank_img,tile_size,model,chan,padding,minval,maxval,batch_size)
T2 = create_image2(img,
                   blank_img,
                   blank_img2,
                   tile_size,
                   model,
                   minval,
                   maxval,
                   batch_size,
                   target_layer1,
                   target_layer2,
                   generate_heatmap,
                   heatmap_algo,
                   normalize_heatmap,
                   generate_rgba,
                   rgba_out)
print((big_y*big_x)," BLOCKS PROCESSED IN ",round(T2,3), ' SECONDS')
print("TOTAL TIME PER BLOCK : ", (T2/(big_y*big_x)),"sec")

TOTAL BLOCKS :  132912
BATCH SIZE :  100


In [ ]:
total_trees = str(np.ndarray.sum(blank_img))
m = tile_size/10
area = round(((big_y*big_x)*(m*m)),2)
print("TOTAL TREES PREDICTED WITH TILE SIZE " +str(tile_size)+" : " + total_trees)  
print("(COL, ROW)",blank_img.shape)
print("TOTAL AREA PROCESSED : ",area, "m2 --> ",round(area/10000,2), "ha")
print("TOTAL PROCESSING TIME : ",round(T2/60,1),"MINUTES")
print("PREDICTED RANGE : ",blank_img.min()," to ",blank_img.max(), " TREES")

plt.imshow(blank_img) #remove this if image is very large

In [ ]:
total_trees = str(np.ndarray.sum(blank_img))
plt.imshow(blank_img2) #remove this if image is very large
print("TOTAL TREES PREDICTED WITH TILE SIZE " +str(tile_size)+" : " + total_trees)  
print(blank_img.shape)
blank_img.min(),blank_img.max()

## Write Output

In [ ]:
#write FSD map
tiff.imwrite(str(os.path.join(out_path+site+"_totaltrees_"+total_trees+'.tif')),blank_img)

In [ ]:
#write activation map if requested
if generate_heatmap == True:
    if normalize_heatmap == "local":
        tiff.imwrite(str(os.path.join(out_path+site+"ts"+str(tile_size)+"_heatmap_localNorm_"+str(heatmap_algo)+'.tif')),blank_img2)
    elif normalize_heatmap == "global":
        n_heatmap = normalize_activations(blank_img3)
        tiff.imwrite(str(os.path.join(out_path+site+"ts"+str(tile_size)+"_heatmap_globalNorm_"+str(heatmap_algo)+'.tif')),n_heatmap)
    elif normalize_heatmap == "none":
        tiff.imwrite(str(os.path.join(out_path+site+"ts"+str(tile_size)+"heatmap_noNorm_"+str(heatmap_algo)+'.tif')),blank_img2) 